In [1]:
import pandas as pd
from IPython.display import display

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
from scipy.sparse import coo_matrix
from sklearn.feature_extraction.text import TfidfVectorizer 
import csv





def mark_click(val):
    return '<img src="{}" width=50></img>'.format(val)



def search_query(query,vectorizor):
    #process the qurey as we did with ours titles
    processed_query=re.sub('[^a-zA-Z0-9]','',query.lower())
    #get the TFidf score
    score=vectorizor.transform([processed_query])
    #search-using cosine transfrom

    similarity=cosine_similarity(score,tfidf).flatten()
    # similarity is cosine score of all tiltes and current query

    #now to get the 3 highest values of cosine similarity

    indices=np.argpartition(similarity,-3)[-3:]

    #result we will get the actual titles

    result=title.iloc[indices]
    #print(result)
    result=result.sort_values('ratings',ascending=False)
    
    #use result of collbrative filtering 
    top_recs=pd.read_csv('top_recs.csv')
    top_recs.drop(['mod_title', 'mean','count','adjusted_count','score'], axis=1, inplace=True)
    result.drop(['mod_title'], axis=1, inplace=True)
    result = pd.concat([result,top_recs])
    result.drop(columns=result.columns[-1], axis=1, inplace=True)
    result.reset_index(drop=True,inplace=True)
    return result.head(10).style.format({'cover_image':mark_click})


def collabrative_filtering(id):
    #print('cf')
    id=int(id)
    r = pd.read_csv( 'goodbooks/ratings.csv' )
    
    
    #Finding out the books liked by the user
    test=r.loc[pd.to_numeric(r['rating'])>3]
    liked_books=test.groupby('user_id')['book_id'].apply(set)#.reset_index(name="liked")
    my_books=test.groupby('user_id')
    books_titles = pd.read_json("books_titles.json")
    liked_book_id=liked_books[id]
    books_titles['book_id']=books_titles['book_id'].astype(str)
    my_books=books_titles[pd.to_numeric(books_titles['book_id']).isin(liked_book_id)].copy()
    my_books["book_id"] = my_books["book_id"].astype(str)
    user=[id]*len(my_books)
    my_books.insert(0,'user_id',user)
    book_set = set(my_books["book_id"])
    
    #Finding out the people who also like same books as the user
    overlap_users = {}

    with open( 'goodbooks/ratings.csv', 'r') as f:
        csvReader = csv.DictReader(f)

        # convert each csv row into python dict
        book_titles=[]
        for l in csvReader:
            if l['book_id'] in book_set:
                if l['user_id'] not in overlap_users:
                    overlap_users[l['user_id']] = 1
                else:
                    overlap_users[l['user_id']] += 1
    filtered_overlap_users = set([k for k in overlap_users if overlap_users[k] > my_books.shape[0]/5])
    interactions_list = []

    with open('goodbooks/ratings.csv', 'r') as f:

        csvReader = csv.DictReader(f)

        # convert each csv row into python dict
        for l in csvReader:
            if l['user_id'] in filtered_overlap_users:

                interactions_list.append([l['user_id'],l['book_id'],l['rating']])
                
    interactions = pd.DataFrame(interactions_list, columns=["user_id", "book_id", "ratings"])
    interactions = pd.concat([my_books[['user_id','book_id', "ratings"]], interactions])
    interactions["book_id"] = interactions["book_id"].astype(str)
    interactions["user_id"] = interactions["user_id"].astype(str)
    interactions["ratings"] = pd.to_numeric(interactions["ratings"])
    interactions["user_index"] = interactions["user_id"].astype("category").cat.codes
    interactions["book_index"] = interactions["book_id"].astype("category").cat.codes
 

    ratings_mat_coo = coo_matrix((interactions["ratings"], (interactions["user_index"], interactions["book_index"])))
    ratings_mat = ratings_mat_coo.tocsr()
    interactions[interactions["user_id"] == str(id)]
    my_index = 0
    similarity = cosine_similarity(ratings_mat[my_index,:], ratings_mat).flatten()
    indices = np.argpartition(similarity, -15)[-15:]
    similar_users = interactions[interactions["user_index"].isin(indices)].copy()
    similar_users = similar_users[similar_users["user_id"]!=str(id)]
    book_recs = similar_users.groupby("book_id").ratings.agg(['count', 'mean'])
    
    books_titles = pd.read_json("books_titles.json")
    books_titles["book_id"] = books_titles["book_id"].astype(str)
    
    book_recs = book_recs.merge(books_titles, how="inner", on="book_id")
    
    book_recs["adjusted_count"] = book_recs["count"] * (book_recs["count"] / book_recs["ratings"])
    book_recs["score"] = book_recs["mean"] * book_recs["adjusted_count"]
    book_recs = book_recs[~book_recs["book_id"].isin(my_books["book_id"])]
    my_books["mod_title"] = my_books["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True).str.lower()
    my_books["mod_title"] = my_books["mod_title"].str.replace("\s+", " ", regex=True)
    book_recs = book_recs[~book_recs["mod_title"].isin(my_books["mod_title"])]
    book_recs = book_recs[book_recs["mean"] >=4]
    book_recs = book_recs[book_recs["count"]>2]
    top_recs = book_recs.sort_values("mean", ascending=False)
    top_recs=top_recs[:50]
    top_recs.to_csv('top_recs.csv')


if __name__=="__main__":
    title=pd.read_json('books_titles.json')

    #to make search easier we conver titles into TF-IDF score



    vectorizor=TfidfVectorizer()

    tfidf=vectorizor.fit_transform(title['mod_title'])

    # Query is name of the book
    query='SalttotheSea'
    
    #user id
    id=2
    
    # Gives us top recommendations of the books the user shuold read
    collabrative_filtering(id)
    
    # search for the book_name then use the collabrative filtering result to give personalised search experience
    df=search_query(query, vectorizor)
    


ValueError: Expected object or value

In [4]:
df

,book_id,title,ratings,cover_image
0,3228,Salt to the Sea,444,
1,3227,The Signal and the Noise: Why So Many Predictions Fail - But Some Don't,324,
2,10000,The First World War,111,
3,1,The Hunger Games,22806,
4,258,La sombra del viento,3531,
5,39,A Game of Thrones,10692,
6,384,Frostbite,2219,
7,325,A Prayer for Owen Meany,3868,
8,323,Nesnesitelná lehkost bytí,3120,
9,311,The Thorn Birds,3721,
